In [1]:
from IPython.display import display, HTML
display(HTML('<h1><img src="logo.png" style="background-color: black"></h1>'))

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Annual Prediction 

In [3]:
train2 = pd.read_csv("Bihis 2020.csv")

In [4]:
train1 = pd.read_csv("Bihis 2021.csv")

In [5]:
test = pd.read_csv("Bihis 2022.csv")

In [6]:
train2_len = len(train2)
train1_len = len(train1)
test_len = len(test)

In [7]:
df = pd.concat([train2, train1, test], axis=0)
pd.set_option('display.max_rows', df.shape[0]+1)
df.reset_index(drop=True, inplace=True)
df

,Name,Size,Cost,Category,Sales,Year
0,Kamias,Medium,100,Tree,3400.0,2020
1,Kalamasi,Medium,120,Tree,4080.0,2020
2,Papaya,Small,50,Tree,1700.0,2020
3,Tundan,Small,80,Tree,3200.0,2020
4,Lakatan,Small,150,Tree,10500.0,2020
5,Rambutan,Medium,250,Tree,5000.0,2020
6,Suha,Small,130,Tree,1950.0,2020
7,Atis,Medium,130,Tree,3770.0,2020
8,Guyabano,Medium,100,Tree,2200.0,2020
9,Lemon,Large,100,Tree,1700.0,2020


In [8]:
%%capture
print(len(train2), len(train1), len(test), len(df))

In [9]:
%%capture
df.apply(lambda x: len(x.unique()))

In [10]:
%%capture
df.isnull().sum()

In [11]:
%%capture
cat_col = []
for x in df.dtypes.index:
    if df.dtypes[x] == 'object':
        cat_col.append(x)
cat_col

In [12]:
%%capture
cat_col.remove('Name')
cat_col

In [13]:
%%capture
for col in cat_col:
    print(col)
    print(df[col].value_counts())
    print();

In [14]:
%%capture
size_mode = df.pivot_table(values = 'Size', columns = 'Category', aggfunc=(lambda x: x.mode()[0]))
size_mode

In [15]:
miss_bool = df['Size'].isnull()
df.loc[miss_bool, 'Size'] = df.loc[miss_bool, 'Category'].apply(lambda x: size_mode[x])

In [16]:
%%capture
df['Size'].isnull().sum()

In [17]:
df['Total_Year'] = 2022 - df['Year']

In [18]:
%%capture
df['Total_Year']

In [19]:
%%capture
df = pd.get_dummies(df, columns=[ 'Size', 'Category'])
df.head();

In [20]:
%%capture
df.tail();

In [21]:
%%capture
train2 = df.iloc[:train2_len,:]
train1 = df.iloc[train2_len:,:]
test = df.iloc[train2_len:,:]

In [22]:
%%capture
test = test.drop(columns=['Sales'])
test.head();

In [23]:
%%capture
test.tail();

In [24]:
%%capture
X = train2.drop(columns=['Year', 'Name', 'Sales'])
y = train2['Sales']

# Accuracy of model

In [25]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
def train2(model, X, y):
    model.fit(X, y)
    
    pred = model.predict(X)
    
    cv_score = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=5)
    cv_score = np.abs(np.mean(cv_score))
    print("Model Report")
    print("Accuracy:", mean_squared_error(y,pred))
    print("CrossValidationScore:", cv_score)

In [26]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
train2(model, X, y)
coef = pd.Series(model.feature_importances_, X.columns).sort_values(ascending=False)

Model Report
Accuracy: 779406.3972077484
CrossValidationScore: 5331866.14851574


In [27]:
%%capture
x_test = test.drop(columns=['Year', 'Name'])
x_test.head();

In [28]:
%%capture
model = RandomForestRegressor()
model.fit(X, y)

In [29]:
pred = model.predict(x_test)
pred

array([2784.99761905, 3842.26666667, 1838.81666667, 2720.63083333,
       7836.56642857, 3431.        , 2239.78392857, 3667.86666667,
       2784.99761905, 1529.2552381 , 2239.78392857, 2784.99761905,
       2702.57857143, 2239.78392857, 3068.8       , 1529.2552381 ,
       2239.78392857, 3431.        , 3054.        , 6160.44761905,
       4664.5       , 4896.2       , 2416.28095238, 2168.21428571,
       2909.6       , 2376.8       , 1711.2       , 2470.8       ,
       7859.14142857, 3068.8       , 2784.99761905, 3842.26666667,
       1838.81666667, 2720.63083333, 7836.56642857, 3431.        ,
       2239.78392857, 3667.86666667, 2784.99761905, 1529.2552381 ,
       2239.78392857, 2784.99761905, 2702.57857143, 2239.78392857,
       3068.8       , 1529.2552381 , 2239.78392857, 3431.        ,
       3054.        , 6160.44761905, 4664.5       , 4896.2       ,
       2416.28095238, 2168.21428571, 2909.6       , 2376.8       ,
       1711.2       , 2470.8       , 7859.14142857, 3068.8    

# Annual Prediction result

In [30]:
mission = pd.DataFrame()
mission['Name'] = test["Name"]
mission['sales'] = pred
mission['Year'] = 2022
pd.set_option('display.max_rows', df.shape[0]+1)
mission


,Name,sales,Year
28,Kamias,2784.997619,2022
29,Kalamasi,3842.266667,2022
30,Papaya,1838.816667,2022
31,Tundan,2720.630833,2022
32,Lakatan,7836.566429,2022
33,Rambutan,3431.000000,2022
34,Suha,2239.783929,2022
35,Atis,3667.866667,2022
36,Guyabano,2784.997619,2022
37,Lemon,1529.255238,2022


# Quarterly Prediction

In [31]:
mission = pd.DataFrame()
mission['Name'] = test["Name"]
mission['sales'] = pred / 12
mission['Quarterly'] = 3
pd.set_option('display.max_rows', df.shape[0]+1)
mission

,Name,sales,Quarterly
28,Kamias,232.083135,3
29,Kalamasi,320.188889,3
30,Papaya,153.234722,3
31,Tundan,226.719236,3
32,Lakatan,653.047202,3
33,Rambutan,285.916667,3
34,Suha,186.648661,3
35,Atis,305.655556,3
36,Guyabano,232.083135,3
37,Lemon,127.437937,3


# Monthly Prediction

In [32]:
mission = pd.DataFrame()
mission['Name'] = test["Name"]
mission['sales'] = pred / 36
mission['monthly'] = 1
pd.set_option('display.max_rows', df.shape[0]+1)
mission

,Name,sales,monthly
28,Kamias,77.361045,1
29,Kalamasi,106.729630,1
30,Papaya,51.078241,1
31,Tundan,75.573079,1
32,Lakatan,217.682401,1
33,Rambutan,95.305556,1
34,Suha,62.216220,1
35,Atis,101.885185,1
36,Guyabano,77.361045,1
37,Lemon,42.479312,1
